#  *** image source folder 필요

#  텍스트기반 경제사회 빅데이터 분석 기말보고서 

##  - 교통약자 담화데이터 분석 및 이를 활용한 chat-bot 개발 - 

$$ 산업데이터엔지니어링학과 $$
$$ 2022153482 $$
$$ 나선옥 $$

### 개요
*  장애인의 이동권에 대한 문제는 주요한 사회적 문제로 거론되어져 왔음
* 정부는 교통약자의 이동권 보장을 위해 ‘교통약자의 이동편의 증진법’을 제정 지역별로 이동지원 센터를 설립하여 운영
* 연구에 의하면 "교통정보를 취득하기 어렵다"는 점이 특수교통수단 이용에 큰 어려움으로 작용

### 연구목적
* 복지 분야 콜센터 데이터를 활용하여 교통약자를 위한 chat-bot 개발
* chat-bot 이용시 기존 24시간 콜센터 운영에 대한 부담을 감소시킬 수 있음(상담사 측면)
* chat-bot을 통해 정보를 보다 편하게 습득할 수 있음(사용자 측면)

### 데이터
* 데이터는 복지 분야 콜센터 상담데이터를 활용
* 2022-07-12에 공개된 데이터
* 광역이동지원센터, 대학 병원, 정신건강 복지센터에서 14,437건의 상담에  
  대한 2,276,065 문장을 음성 데이터와 label 데이터로 구축
* 전체 데이터는 각 상담유형과 상담 주제에 따라 분류
* 본 과제에서는 광역이동지원 센터에 대한 데이터만 사용 

<img src=".\image source\그림1.png" style="float: center; width:400px; height:200px;">
<div style="clear: both;"></div>
$$ [그림1-광역이동지원센터-데이터-통계] $$

* 민원 데이터는 chat-bot으로 사용이 어려워 제외  
* 상담과 고객데이터만 활용(약33만 문장)

### 데이터 스키마 정의
* chat-bot은 user action, intent, slot을 필요로 함
* 본 데이터는 이 3가지 모두 정의되어 있지 않음
* 33만개의 데이터를 모두 살펴보고 정의하는 것은 시간소모가 큼
* LDA와 Sentence BERT를 활용하여 전체 문장을 clustering하고, 이를 바탕으로 class 정의
* LDA의 경우 문장당 포함된 단어가 적거나 없는 경우가 많아 분석에 크게 도움이 되지 않음
* Sentence BERT의 경우 유사한 의도나 액션을 가지고 있는 경우를 잘 잡아주어 이를 근거로 정의
* slot은 실제 상담사가 답변을 위해 요청하는 정보를 근거로 정의
* SGD dataset을 밴치마크하여 스키마를 정의


<img src=".\image source\그림2.png" style="float: center; width:800px; height:200px;">
<div style="clear: both;"></div>
$$ [그림2-Intent-정의] $$

* 사용자의 의도 중, 상담사가 해결할 수 있는 경우를 정의  
* 답변에 필요한 required slot을 같이 정의

<img src=".\image source\그림4.png" style="float: center; width:550px; height:250px;">
<div style="clear: both;"></div>
$$ [그림3-Action-정의] $$

* 원천 데이터의 상담사 담화가 완전한 1대1 구조가 아니기에 시스템의 Intent가 필요한 경우 제외  

<img src=".\image source\그림3.png" style="float: center; width:500px; height:180px;">
<div style="clear: both;"></div>
$$ [그림4-slot-정의] $$

* slot이 required slot과 매칭되지 않는 부분은 Dialog State Tracker 구현시 보정


### Annotation
* 모든 문장을 Annotation 하기에는 너무 많아 20문장이내의 담화문을 대상으로 선정
* 사용자 발화기준 3,000문장에 대해서 Annotation 수행
* 데이터에 대한 통계는 기말보고서 참조

### 실험결과
* model = KoElectra_base_v3 / Electra를 한국어에 맞춰 재학습한 모델(https://github.com/monologg/KoELECTRA)
* train = 0.9, test = 0.1
* epoch = 100

<img src=".\image source\그림5.png" style="float: center; width:500px; height:230px;">
<div style="clear: both;"></div>
$$ [그림5- Classification(intent)] $$

* 예약변경 및 취소, 등록과 관련된 부분이 떨어짐
* 이를 개선하기 위해 데이터 증강방법을 사용하였으나, f1-score 0.5~1.0% 하락
* 짧은 문장에 단어를 대치하는 기법을 적용하여 문맥적인 의미에 큰 변화를 준 것이 원인

<img src=".\image source\그림6.png" style="float: center; width:500px; height:280px;">
<div style="clear: both;"></div>
$$ [그림6- Classification(action)] $$

* 평가 데이터가 많지 않아 정확한 분석은 어렵지만 전체적으로 양호한 결과를 보이고 있음

<img src=".\image source\그림7.png" style="float: center; width:500px; height:280px;">
<div style="clear: both;"></div>
$$ [그림7- NER(slot)] $$

* 비식별화된 단어(oo복지관 등)를 최대한 실제에 가깝게 대치
* 대치하는 단어의 분포가 충분하지 않아 과적합이 발생한 것으로 보임
* support의 수 차이는 표에서 outside를 제거했기 때문
* 정확한 평가를 위해 token단위로 평가

### demo
* retrieval chat-bot과 일반적인 TOD를 조합한 형태로 구현
* 정보를 제공하는 경우 FAQ를 자체 구축하고, 이를 SBERT로 유사도 구하는 방식으로 구현(retrieval chat-bot)
<img src=".\image source\그림8.png" style="float: center; width:800px; height:200px;">
<div style="clear: both;"></div>
$$ [그림8-retrieval-chat-bot-Architecture] $$


* 차량배차예약, 예약 변경 및 취소는 시나리오 기반 TOD로 구축
* 간단한 형태로 DST(Dialog State Tracker)와 DP(Dialogue policy)를 구현
* 사용자의 Intent를 파싱하고, 해당 Intent에 대한 required slot을 요청하는 문장을 반환, slot filling이 끝나면 최종 답변을 생성하여 출력(일반적인 TOD)

<img src=".\image source\그림9.png" style="float: center; width:600px; height:400px;">
<div style="clear: both;"></div>
$$ [그림9-TOD-Demo-차량배차의-경우] $$

<img src=".\image source\그림10.png" style="float: center; width:800px; height:150px;">
<div style="clear: both;"></div>
$$ [그림10-retrieval-chat-bot의-경우] $$

### 결론
* 복지데이터를 원천데이터로 하여 교통약자를 위한 이동지원센터 챗봇을 개발
* 상담 상황에서 발생할 수 있는 user intent, action, slot를 정의하고, 이를 annotation
* 실험결과에서 나쁘지 않은 결과를 얻었지만, intent의 수가 적어 이에 대한 데이터 증강 수행
* 사용자와의 대화상황을 가정하여 간단한 시나리오를 작성
* 위에서 작성한 시나리오를 기반하여 챗봇을 개발
* 전체 데이터가 많음에도 이를 잘 살리지 못한것이 아쉬움으로 남음
* 데모 시스템 차원에서는 사용자 액션을 구체적으로 설정한 만큼 의도가 적어졌는데 이 액션을 잘 활용하지 못해 성능이 부족해진것이 아닌가 생각됨
* 액션과 의도를 모두 모델을 통해 분류하는데 액션을 규칙에 기반해서 정의했다면 조금 더 활용할 방향이 많지 않았을까 생각됨